In [1]:
from pathlib import Path
import json
import numpy as np
import pandas as pd


# paragraph

In [2]:
def get_results(path_folder, name):
    ap_list = []
    roc_list = []
    records=[]
    for file in path_folder.rglob("*/summary_dict.json"):
        with open(file) as f :
            summary_data=json.load(f)
        if not (file.parents[0]/Path("test_set_results_dict.json")).exists():
            continue
        result_dict = file.parents[0]/Path("test_set_results_dict.json")
        with open(result_dict) as f :
            result_data = json.load(f)
        summary_data["ap"]=ap=float(result_data["ap"])
        summary_data["auc"]=auc=float(result_data["auc"])
        summary_data["pen"]=file.parents[0].as_posix().split("_")[6]
        summary_data["mcc_best_val"]=float(result_data.get("mcc_best_val",0))
        summary_data["f1_best_val"]=float(result_data.get("f1_best_val",0))
        summary_data["mcc_05"]=float(result_data.get("mcc_05",0))
        summary_data["f1_05"]=float(result_data.get("f1_05",0))
        summary_data["mcc"]=float(result_data.get("mcc",0))
        records.append(summary_data)
        ap_list.append(ap)
        roc_list.append(auc)
    df=pd.DataFrame.from_records(records).drop(columns=["best_epoch",
        "result_folder",
        "best_ap",
        "train_folder_path",
        "val_folder_path",
        "n_epochs",
        "input_size",
        "override",
        "patience",
        ])
    df.groupby(["pen","mask_prob","alphas","embedding_models", "batch_size","dims","dropouts","positive_weight","learning_rate",])[["ap","auc", "mcc_best_val","mcc_05","f1_05","mcc","f1_best_val"]].mean().to_csv(path_folder.parents[0]/Path("results")/Path(f"{name}.csv"))


In [3]:
path_folder = Path("/home/athenes/benchmark2/mipe/250106")
get_results(path_folder, "250106")


In [4]:
path_folder = Path("/home/athenes/benchmark2/paragraph/250106")
get_results(path_folder, "250106")


In [5]:
path_folder = Path("/home/athenes/benchmark2/pecan/250104")
get_results(path_folder, "250104")


In [6]:
path_folder = Path("/home/athenes/benchmark2/paragraph/ablation_study_2")
get_results(path_folder, "ablation_study_2")
